In [117]:
import hashlib
import math
import random
import numpy as np
import time

# Code pour une detection simple de collision

In [118]:
class Statistics:
    n_points = 0
    n_trails = 0
    n_cycles = 0

In [119]:
def trail(fhash,  k , l):
    """
    retourne un triplet (x0, xd, d) 
    fhash: une fonction d'hachage quelconque (md5, sha256)
    k: longueur de la chaine générer
    l: condition d'arret pour le point distingué : les l derniers bits doit etre nul
    """
    x0 = random.getrandbits(k)
    tmp = x0 # c'est un int
    d = 0
    max_it = 20/ (1/(2**l))  # d'apres l'article
    # 2**(l+2)
    mask_k = 2**k - 1
    mask_l = 2**l - 1
    lbyte = math.ceil(l/8)
    while True:
        if d == max_it:
            #print("Risque de cycle")
            return None
        
        if tmp & mask_l == 0:  # condition d'arret
            xd = tmp
            break
        d += 1
        x = tmp.to_bytes(16, byteorder='big')  
        y = fhash(x).digest()
        Statistics.n_points += 1
        tmp = int.from_bytes(y, byteorder='big') & mask_k
        #print(d, tmp)
    
    return (x0, xd, d)

In [ ]:
def collision_detection(fhash, k, l):
    """
    detecte une seule collision
    retourne le couple de triplet ( (x0,xd,d), (x0',xd,d') )
    """
    dico = {}
    while True: 
        res = trail(fhash, k, l)
        Statistics.n_trails += 1
        if res == None:
            Statistics.n_cycles += 1
            continue
            
        x0, xd, d = res
        
        if xd in dico:
            print("Collision found")
            return ( (x0,xd,d),(dico[xd][0], xd,dico[xd][1])  )  
        
        dico[xd] = (x0, d)

In [88]:
k = 50
l = 10
log_nF = k/2 + 0.5*math.log(math.pi/2)
print("On s'attend à 2^{:.1f} évaluations de F".format(log_nF))
print("On s'attend à 2^{:.1f} trails".format(log_nF - l))
debut = time.time()
print(collision_detection(hashlib.md5, 50, 10))
fin = time.time()
print("Temps : {:.1f}s".format(fin - debut))

On s'attend à 2^25.2 évaluations de F
On s'attend à 2^15.2 trails
Collision found
((163275221851853, 552532737869824, 2225), (155129268956935, 552532737869824, 3029))
Temps : 73.7s


In [89]:
print("--> 2^{:.1f} points".format(math.log(Statistics.n_points, 2)))
print("--> 2^{:.1f} trails".format(math.log(Statistics.n_trails, 2)))
print("--> 2^{:.1f} cycles".format(math.log(Statistics.n_cycles, 2)))

--> 2^26.3 points
--> 2^16.4 trails
--> 2^10.6 cycles


In [248]:
def trace_k(fhash, k , l):
    x_taille_graph = np.linspace(10,k,k-10,dtype=int)
    y_temps = np.zeros(k-10)
    tmp= np.zeros(5)
    for i in range(len(x_taille_graph)):
        
        for j in range(5):
            debut = time.time()
            collision_detection(fhash ,x_taille_graph[i] , l)
            fin = time.time()
            tmp[j] = fin-debut
        y_temps[i] = np.mean(tmp)

    plt.plot(x_taille_graph, y_temps, color= np.random.rand(3,), label=fhash.__name__)
    plt.gca().legend().set_visible(True)
    plt.title(fhash.__name__)
    plt.xlabel("k")
    plt.ylabel("temps de calcul (s)")
    plt.show()
    
def trace_l(fhash, k , l):
    x_taille_graph = np.linspace(2,l,l-2,dtype=int)
    y_temps = np.zeros(l-2)
    
    for i in range(len(x_taille_graph)):
        debut = time.time()
        collision_detection(fhash ,k , x_taille_graph[i])
        fin = time.time()
        y_temps[i] = fin-debut

    plt.plot(x_taille_graph, y_temps, color= np.random.rand(3,), label=fhash.__name__)
    plt.gca().legend().set_visible(True)
    plt.title(fhash.__name__)
    plt.xlabel("l")
    plt.ylabel("temps de calcul (s)")
    plt.show()
